# **FED Text Score Software (Beta version)**

We acknowledge the support of Elia Landini, Jessie Cameron & Lina Avril (Pantheon-Sorbonne University) in the development of this project.

### **Introduction**

The project aims to conduct textual analysis of the American Federal Reserve's (FED) Monetary Policy Statements through the deployment of Python-based software. These monetary policy decisions take place every six months and following the meeting, the Federal Reserve Board explain eventual decisions to the Congress at the press conference and answer questions from journalists. Firstly, we will develop a web scraping script to extract textual data from the FED's website. Subsequently, we'll use the Natural Language Toolkit (NLTK) package to preprocess the text, including tokenization, stemming, and converting words to lowercase. Next, the Loughran McDonald Sentiment Dictionary will be employed to transform the cleaned qualitative text data into a quantitative measure of the FED's communication tone. This communication measure will then be regressed against the output gap and inflation gap, obtained via API, to assess the sensitivity of the FED's communication to these macroeconomic variables. Throughout the project, we'll employ various visualization and analysis packages to explore the data and conduct preliminary analysis. Finally, we plan to develop a user-friendly interface for easy access and interpretation of our findings.

### **Preliminary steps**

In [149]:
!pip install pandas
!pip install matplotlib
!pip install requests-html
!pip install seaborn
!pip install numpy
!pip install schedule
!pip install statsmodels
!pip install reportlab
!pip install scipy
!pip install linearmodels
!pip install openai
!pip install fredapi

### **FED web scraping**

In [145]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

In [232]:
# FED scraping function
# With the following function, we aim to retrieve and filter text-based sources concerning monetary policy decisions undertaken by the FED itself and released to the public on semiannual press conferences.  
# Customize the function to scrape articles from the ECB/Eurostystem website within the folder named "Monetary Policy Report"
# The function is also designed to include filtering options to select specific text-based sources according to topic and typology of the publication. However in our specific case we will be interesented only in semiannual reports concerning monetary policy.
# Base URL-FED: https://www.federalreserve.gov/monetarypolicy/publications/mpr_default.htm    

def fed_get_articles(topic, publication_type, sub_class):
    
    # Base URL settings 
    base_url = f"https://www.federalreserve.gov/{topic}/{publication_type}/mpr_default.htm"
    base_domain = f"https://www.federalreserve.gov"
    
    article_urls = []

    # From the base URL, we now extract all the available URLs on the page by deploying Request and BeatifulSoup packages 
    response = requests.get(base_url)
    soup = BeautifulSoup(response.content, "html.parser")
    print("Scraping URL:", base_url)

    # Find and filter article URLs, ruling out other structural URL non-inherent to the analysis
    # We are also interested only in full report publications and not summaries. In our case this difference is highlighted by the subclass "testimony" in which full report publications are stored
    # It is also worth noticing that the publications' URL has slightly changed throughout time (from 2016 onwards)
    for link in soup.find_all("a", href=True):
        article_url = link["href"]

        # To limit the research to the first 5 results, we may want to activate this loop
        # if len(article_urls) >= 5:  
            # break
        if not article_url.startswith("http"):
            article_url = base_domain + article_url
        
        if f"{sub_class}" in article_url:
            print(article_url)
            article_urls.append(article_url)

    # Delete the first element of the article_urls list representing the general folder    
    for url in article_urls:     
        if url == f"https://www.federalreserve.gov/newsevents/{sub_class}.htm":
            article_urls.remove(url)
            break

    return article_urls

In [233]:
# Retrieving FED semiannual reports' URLs through the previous function by specifying the value of each parameter to fit our research scope
topic = "monetarypolicy"
publication_type = "publications"
sub_class = "testimony"

print(fed_get_articles(topic, publication_type, sub_class))

Scraping URL: https://www.federalreserve.gov/monetarypolicy/publications/mpr_default.htm
https://www.federalreserve.gov/newsevents/testimony.htm
https://www.federalreserve.gov/newsevents/testimony/powell20240306a.htm
https://www.federalreserve.gov/newsevents/testimony/powell20230307a.htm
https://www.federalreserve.gov/newsevents/testimony/powell20230621a.htm
https://www.federalreserve.gov/newsevents/testimony/powell20220302a.htm
https://www.federalreserve.gov/newsevents/testimony/powell20220622a.htm
https://www.federalreserve.gov/newsevents/testimony/powell20210223a.htm
https://www.federalreserve.gov/newsevents/testimony/powell20210714a.htm
https://www.federalreserve.gov/newsevents/testimony/powell20200211a.htm
https://www.federalreserve.gov/newsevents/testimony/powell20200616a.htm
https://www.federalreserve.gov/newsevents/testimony/powell20190226a.htm
https://www.federalreserve.gov/newsevents/testimony/powell20190710a.htm
https://www.federalreserve.gov/newsevents/testimony/powell20180

### **Text Retrieving**

### **FED Data Import**

In [150]:
import os
import fredapi as fa
import pandas as pd
from fredapi import Fred

In [151]:
# Freadpi settings
# DISCLAIMER: the so API key is strictly personal and you should refrain from sharing it with strangers 

# FRED API key  setting
# you can generate an API key on https://fred.stlouisfed.org/docs/api/api_key.html by registrating a valid account
# FRED API key-example: a8f0cdf1d0b64d1188b9c64ed64c77b6

fred = Fred(api_key="a8f0cdf1d0b64d1188b9c64ed64c77b6")

1) EXAMPLE: Japanese Yen to U.S. Dollar Spot Exchange Rate 

In [152]:
# Retrieving data for Japanese Yen to U.S. Dollar Spot Exchange Rate 
# https://fred.stlouisfed.org/series/EXJPUS 
# frequency: monthly
# time span (year): 1971-2024
# dataframe tag: yen

dfyen = fred.get_series("EXJPUS")
dfyen = dfyen.to_frame()
print(dfyen.head())

                   0
1971-01-01  358.0200
1971-02-01  357.5450
1971-03-01  357.5187
1971-04-01  357.5032
1971-05-01  357.4130


In [153]:
# Data manipulation 

# get the columns names 
dfyen_column_names = dfyen.columns
print([dfyen_column_names])

# Rename columns to better fit our vocabulary
dfyen1 = dfyen.rename(columns={dfyen.columns[0]: "Yen"})

# Reset the index, making the time index a regular column and assign to that new column the name "Year"
dfyen1.reset_index(inplace=True) 
dfyen1.rename(columns={"index": "Month"}, inplace=True) 
dfyenX = dfyen1
print(dfyenX)

[RangeIndex(start=0, stop=1, step=1)]
         Month       Yen
0   1971-01-01  358.0200
1   1971-02-01  357.5450
2   1971-03-01  357.5187
3   1971-04-01  357.5032
4   1971-05-01  357.4130
..         ...       ...
635 2023-12-01  143.9815
636 2024-01-01  146.2943
637 2024-02-01  149.6150
638 2024-03-01  149.8186
639 2024-04-01  153.8900

[640 rows x 2 columns]
